In [5]:
# --- 0. Imports and autoreload (optional) ---
import numpy as np
import importlib
import pprint

import sys
import os

# Add parent folder (one level up from notebook) to Python path
sys.path.append(os.path.abspath(".."))


# If your files aren't named exactly like this, change the names below:
VEC_MOD_NAME = "amp_denoising_functions_vectorized"       # e.g., vectorized.py
NONVEC_MOD_NAME = "amp_denoising_functions"  # e.g., non_vectorized.py

vec = importlib.import_module(VEC_MOD_NAME)
base = importlib.import_module(NONVEC_MOD_NAME)

# If you edit the modules and want to re-import during the session:
importlib.reload(vec)
importlib.reload(base)

# --- 1. Helpers to generate random SPD matrices and nice prints ---
def rand_spd_batch(n, jitter=0.2, seed=0):
    rng = np.random.default_rng(seed)
    A = rng.standard_normal((n, 2, 2))
    V = A @ np.swapaxes(A, -1, -2) + jitter * np.eye(2)[None, :, :]
    return V

def max_abs_diff(a, b):
    return float(np.max(np.abs(a - b))) if a.size else 0.0

def make_psd_like(M, eps=1e-3):
    # symmetrize and add small jitter to the diagonal
    M = 0.5*(M + M.transpose(0,2,1))
    M += eps*np.eye(2)[None, ...]
    return M

# --- 2. Test configuration ---
n = 200  # number of samples

rng = np.random.default_rng(123)

# Globals / scalars (you can tweak):
beta_u = 1.0
beta_v = 2.0
beta_tilde_v = vec.beta_tilde(beta_v)  # scalar
gamma = 1.0
delta = 0.0

# Global Q (2x2)
Q = np.array([[0.5, 0.2],
              [0.2, 0.8]])

# Build a gently-conditioned batch
V = rng.normal(scale=0.2, size=(n,2,2))
V = make_psd_like(V, 1e-3)

# From your scalar rules:
c2 = vec.c_2(beta_tilde_v, Q[1,1])        # scalar
c1 = vec.c_1(c2, beta_tilde_v)            # scalar
G  = vec.gamma_matrix(c1)                  # (2,2) global Gamma

# Batched data:
# V = Q - rand_spd_batch(n, jitter=0.01, seed=42)     # (n,2,2)
# Include both classes; ensure at least a few positives to exercise the y==1 branch:
y = rng.choice([-1, 1], size=(n,), p=[0.3, 0.7])
omega = rng.standard_normal((n, 2))             # (n,2)

# --- 3. Compute M, N using vectorized module ---
M, N, V_inv, G_check = vec.compute_M_N_from_V_c1(V, c1, eps=1e-10, symmetrize=True)
assert np.allclose(G, G_check), "Gamma from compute_M_N_from_V_c1 mismatch."

# --- 4. Compute (a, b, c, d) using vectorized integral_parameters ---

a_vec, b_vec, c_vec, d_vec = vec.integral_parameters(
    beta_u, beta_v, omega, Q, N, M, c1, c2
)

print(a_vec)
print(b_vec)
print(c_vec)
print(d_vec)

# --- 5. Vectorized outputs ---
fout_vec  = vec.f_out(beta_u, beta_v, gamma, delta, G, c1, c2, y, omega, Q, N,
                      a_vec, b_vec, c_vec, d_vec)
dfout_vec = vec.df_out(fout_vec, beta_u, beta_v, gamma, delta, G, c1, c2, y, omega, Q, N,
                       a_vec, b_vec, c_vec, d_vec)
fQ_vec    = vec.f_Q(beta_u, beta_v, gamma, delta, beta_tilde_v, c1, c2, y,
                    omega, Q, M, V, a_vec, b_vec, c_vec, d_vec, fout_vec, dfout_vec)

# --- 6. Non-vectorized loop outputs ---
fout_loop  = np.zeros_like(fout_vec)
dfout_loop = np.zeros_like(dfout_vec)
fQ_loop    = np.zeros_like(fQ_vec)

for i in range(n):
    # Per-sample views
    yi      = int(y[i])
    omegai  = omega[i]
    Mi      = M[i]
    Ni      = N[i]
    Vi      = V[i]

    # Per-sample integral parameters from non-vectorized:
    ai, bi, ci, di = base.integral_parameters(
        beta_u, beta_v, omegai, Q, Ni, Mi, c1, c2
    )

    # f_out
    fout_i = base.f_out(beta_u, beta_v, gamma, delta, G, c1, c2, yi, omegai, Q, Ni,
                        ai, bi, ci, di)
    fout_loop[i] = fout_i

    # df_out depends on fout_i
    dfout_i = base.df_out(fout_i, beta_u, beta_v, gamma, delta, G, c1, c2, yi, omegai, Q, Ni,
                          ai, bi, ci, di)
    dfout_loop[i] = dfout_i

    # f_Q depends on fout_i & dfout_i
    fQ_i = base.f_Q(beta_u, beta_v, gamma, delta, beta_tilde_v, c1, c2, yi, omegai, Q, Mi, Vi,
                    ai, bi, ci, di, fout_i, dfout_i)
    fQ_loop[i] = fQ_i

# --- 7. Compare results ---
rtol = 1e-6
atol = 1e-8

def check_and_report(name, A, B):
    same = np.allclose(A, B, rtol=rtol, atol=atol)
    print(f"{name}: {'OK' if same else 'MISMATCH'} | max abs diff = {max_abs_diff(A, B):.3e}")
    return same

ok1 = check_and_report("f_out",  fout_vec,  fout_loop)
ok2 = check_and_report("df_out", dfout_vec, dfout_loop)
ok3 = check_and_report("f_Q",    fQ_vec,    fQ_loop)

if not (ok1 and ok2 and ok3):
    # Optional: show a few problematic indices where y==1 (active path)
    bad_idx = np.where(
        (np.any(~np.isclose(fout_vec,  fout_loop,  rtol=rtol, atol=atol), axis=1)) |
        (np.any(~np.isclose(dfout_vec, dfout_loop, rtol=rtol, atol=atol), axis=(1,2))) |
        (np.any(~np.isclose(fQ_vec,    fQ_loop,    rtol=rtol, atol=atol), axis=(1,2)))
    )[0]
    print("\nIndices with any mismatch (showing up to first 5):", bad_idx[:5])
    for idx in bad_idx[:5]:
        print(f"\n--- i = {idx}, y={y[idx]} ---")
        print("f_out vec vs loop:\n", fout_vec[idx], "\n", fout_loop[idx])
        print("df_out vec vs loop:\n", dfout_vec[idx], "\n", dfout_loop[idx])
        print("f_Q vec vs loop:\n", fQ_vec[idx], "\n", fQ_loop[idx])
else:
    print("\nAll tests passed within tolerances.")


[0.71439929 0.37280398 0.71729046 0.30338544 0.320167   0.94210079
 0.35693967 0.56714693 0.56750308 0.33708737 0.40991198 0.76727406
 0.15544202 0.60921701 0.80173139 0.76282825 0.78704948 0.54025432
 0.3344413  0.66466346 0.6266693  0.81712175 0.42545989 0.42102908
 0.62905115 0.6138914  0.58783732 0.37132627 0.91533875 0.73589695
 0.60125113 0.59439322 0.18794916 0.64365816 0.35303029 0.61870884
 0.42436953 0.49935877 0.80739958 0.53778415 1.03288111 0.31382722
 0.82551163 0.44051323 0.61105509 0.6598743  0.89700166 0.56434272
 0.44737534 0.76670159 0.8172946  0.17158411 1.00957    0.45722457
 0.93877529 0.25924984 0.38181497 0.67607908 0.98169761 1.06283512
 0.46252798 0.7044323  0.6436671  0.34237836 0.80989289 0.42674848
 0.66108286 0.42509747 0.53337231 0.86109817 0.86720218 0.79267475
 0.76701307 0.50089678 0.47395072 0.81265388 0.54289496 0.45778481
 0.15322249 0.7392804  0.90564133 0.97019143 0.41886098 0.60719734
 0.63856121 0.13353781 0.3711436  0.66891085 0.22614201 0.9011

## Old

In [2]:
# %% Imports + your helpers (paste your own versions if already defined above)
import numpy as np

import sys
import os

# Add parent folder (one level up from notebook) to Python path
sys.path.append(os.path.abspath(".."))

# If your function lives in another file, import it here:
from amp_denoising_functions import beta_tilde, c_1, c_2, p_out, Z_out, f_out, df_out, f_Q, gamma_matrix, integral_parameters
from lambda_nu_integrals import int_1, int_2, int_3, int_4, int_5
from state_evolution import generate_latents
from amp_denoising_functions_vectorized import f_out_batched, df_out_batched, f_Q_batched, integral_parameters_batched, c2_from_Q, c1_from_c2, compute_M_N_from_V_c1, gamma_matrix_batched

In [9]:

# ---------- paste your int_1...int_5 definitions above this cell ----------
# (we'll assume they are already defined in the notebook session)

# ---------- paste your f_out_batched, df_out_batched, f_Q_batched above ----------
# (we'll assume they are already defined in the notebook session)

# ---------- paste your scalar f_out, df_out, f_Q above ----------
# (we'll assume they are already defined in the notebook session)

# ---------- test harness ----------
def make_psd_like(M, eps=1e-3):
    # symmetrize and add small jitter to the diagonal
    M = 0.5*(M + M.transpose(0,2,1))
    M += eps*np.eye(2)[None, ...]
    return M

def run_consistency_test(n=128, seed=0, gamma=1.0, delta=0.0,
                         beta_u=1.0, beta_v=2.0, rtol=1e-8, atol=1e-9):
    rng = np.random.default_rng(seed)

    # Build a gently-conditioned batch
    Q = rng.normal(scale=0.2, size=(n,2,2))
    V = rng.normal(scale=0.2, size=(n,2,2))
    Q = make_psd_like(Q, 1e-3)
    V = make_psd_like(V, 1e-3)

    omega = rng.normal(size=(n,2))
    y = rng.choice([-1,1], size=n, p=[0.5, 0.5])

    # Per-sample c2, c1 (depend on Q)
    c2 = c2_from_Q(Q, beta_v)          # (n,)
    # avoid degenerate c2 too close to 0
    c2 = np.where(np.abs(c2) < 1e-3, np.sign(c2)*1e-3, c2)
    c1 = c1_from_c2(c2, beta_v)        # (n,)

    M, N, V_inv, G = compute_M_N_from_V_c1(V, c1)

    # Per-sample integral params (batched)
    bu = np.full(n, beta_u)
    bv = np.full(n, beta_v)
    a_b, b_b, c_b, d_b = integral_parameters_batched(bu, bv, omega, Q, N, M, c1, c2)

    # gamma_mat: consistent with your gamma_matrix(beta_v)
    G = gamma_matrix_batched(c1)

    # -------- Vectorized path --------
    fout_b = f_out_batched(beta_u, beta_v, gamma, delta, G, c1, c2, y, omega, Q, N, a_b, b_b, c_b, d_b)
    dfout_b = df_out_batched(fout_b, beta_u, beta_v, gamma, delta, G, c1, c2, y, omega, Q, N, a_b, b_b, c_b, d_b)
    fQ_b = f_Q_batched(beta_u, beta_v, gamma, delta, beta_tilde(beta_v), c1, c2, y,
                       omega, Q, M, V, a_b, b_b, c_b, d_b, fout_b, dfout_b)

    # -------- Scalar loop path --------
    fout_loop = np.zeros_like(fout_b)
    dfout_loop = np.zeros_like(dfout_b)
    fQ_loop = np.zeros_like(fQ_b)

    for i in range(n):
        c2 = c_2(beta_tilde(beta_v), Q[i, 1, 1])
        c1 = c_1(c2, beta_tilde(beta_v))
        # recompute a,b,c,d per sample using scalar helper (sanity)
        a_i, b_i, c_i, d_i = integral_parameters(beta_u, beta_v, omega[i], Q[i], N[i], M[i], c1, c2)
        # compare with batched params to catch a/b/c/d bugs
        # (loose tol because small numeric symmetrization may differ)
        assert np.allclose(a_i, a_b[i], rtol=1e-12, atol=1e-12)
        assert np.allclose(b_i, b_b[i], rtol=1e-12, atol=1e-12)
        assert np.allclose(c_i, c_b[i], rtol=1e-12, atol=1e-12)
        assert np.allclose(d_i, d_b[i], rtol=1e-12, atol=1e-12)

        

        gamma_mat = gamma_matrix(c1)

        fout_loop[i] = f_out(beta_u, beta_v, gamma, delta, gamma_mat, c1, c2,
                             y[i], omega[i], Q[i], N[i], a_i, b_i, c_i, d_i)
        dfout_loop[i] = df_out(fout_loop[i], beta_u, beta_v, gamma, delta, gamma_mat, c1, c2,
                               y[i], omega[i], Q[i], N[i], a_i, b_i, c_i, d_i)
        fQ_loop[i] = f_Q(beta_u, beta_v, gamma, delta, beta_tilde(beta_v), c1, c2, y[i],
                         omega[i], Q[i], M[i], V[i], a_i, b_i, c_i, d_i, fout_loop[i], dfout_loop[i])

    # -------- Comparisons --------
    def report(name, vec, loop):
        ok = np.allclose(vec, loop, rtol=rtol, atol=atol)
        diff = np.max(np.abs(vec - loop))
        rel = diff / (np.max(np.abs(loop)) + 1e-16)
        print(f"{name:12s} | allclose={ok} | max_abs={diff:.3e} | max_rel={rel:.3e}")

    print(f"n={n}, gamma={gamma}, delta={delta}, beta_u={beta_u}, beta_v={beta_v}")
    report("f_out", fout_b, fout_loop)
    report("df_out", dfout_b, dfout_loop)
    report("f_Q", fQ_b, fQ_loop)

    # optional: assert to stop on failure
    assert np.allclose(fout_b, fout_loop, rtol=rtol, atol=atol)
    assert np.allclose(dfout_b, dfout_loop, rtol=rtol, atol=atol)
    assert np.allclose(fQ_b, fQ_loop, rtol=rtol, atol=atol)

# ---------- run a few modes ----------
for (g,d) in [(1.0,0.0), (0.0,1.0), (0.0,0.0)]:
    run_consistency_test(n=128, seed=42, gamma=g, delta=d, beta_u=1.0, beta_v=2.0, rtol=1e-7, atol=1e-9)


n=128, gamma=1.0, delta=0.0, beta_u=1.0, beta_v=2.0
f_out        | allclose=True | max_abs=4.441e-16 | max_rel=9.294e-17
df_out       | allclose=True | max_abs=1.776e-15 | max_rel=1.594e-16
f_Q          | allclose=True | max_abs=2.220e-16 | max_rel=1.785e-17
n=128, gamma=0.0, delta=1.0, beta_u=1.0, beta_v=2.0
f_out        | allclose=True | max_abs=4.441e-16 | max_rel=1.067e-16
df_out       | allclose=True | max_abs=2.665e-15 | max_rel=6.190e-16
f_Q          | allclose=True | max_abs=8.882e-16 | max_rel=9.172e-17
n=128, gamma=0.0, delta=0.0, beta_u=1.0, beta_v=2.0
f_out        | allclose=True | max_abs=4.441e-16 | max_rel=9.564e-17
df_out       | allclose=True | max_abs=3.553e-15 | max_rel=7.586e-16
f_Q          | allclose=True | max_abs=3.331e-16 | max_rel=2.814e-17
